Baixando dependências

In [12]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib

Importando dependências para o projeto

In [17]:
import cv2 as cv

Função de player de vídeo:

In [14]:
captura = cv.VideoCapture('../videos/parkour.mp4')
if not captura.isOpened():
    print("Error ao abrir o arquivo de vídeo. Finalizando...")
    exit()

while True:
    ret, frame = captura.read()
    if not ret:
        print("Não pode receber frame. Finalizando...")
        break
    # Mostra o frame capturado
    cv.imshow('Frame', frame)
    # Verifica se a tecla 'q' foi pressionada
    if cv.waitKey(30) & 0xFF == ord('q'):
        break
    # Verifica se o botão de fechar (x) foi clicado
    if cv.getWindowProperty('Frame', cv.WND_PROP_VISIBLE) < 1:
        break
    
captura.release()
cv.destroyAllWindows()


Não pode receber frame. Finalizando...


In [18]:

def detectar_movimento(video_path):
    captura = cv.VideoCapture(video_path)

    if not captura.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Lê o primeiro quadro do vídeo
    ret, frame_anterior = captura.read()
    if not ret:
        print("Não foi possível ler o vídeo.")
        return

    # Converte o primeiro quadro para escala de cinza
    frame_anterior = cv.cvtColor(frame_anterior, cv.COLOR_BGR2GRAY)
    frame_anterior = cv.GaussianBlur(frame_anterior, (21, 21), 0)

    while True:
        # Lê o próximo quadro
        ret, frame_atual = captura.read()
        if not ret:
            break

        # Converte o quadro atual para escala de cinza
        frame_atual_gray = cv.cvtColor(frame_atual, cv.COLOR_BGR2GRAY)
        frame_atual_gray = cv.GaussianBlur(frame_atual_gray, (21, 21), 0)

        # Calcula a diferença absoluta entre o quadro atual e o anterior
        diferenca = cv.absdiff(frame_anterior, frame_atual_gray)

        # Aplica uma limiarização (threshold) para destacar áreas de movimento
        _, limiar = cv.threshold(diferenca, 25, 255, cv.THRESH_BINARY)

        # Dilata o resultado para cobrir buracos
        limiar = cv.dilate(limiar, None, iterations=2)

        # Encontra os contornos das áreas com movimento
        contornos, _ = cv.findContours(limiar, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        # Desenha retângulos ao redor dos contornos detectados
        for contorno in contornos:
            if cv.contourArea(contorno) < 500:  # Ignora pequenas áreas de movimento
                continue
            (x, y, w, h) = cv.boundingRect(contorno)
            cv.rectangle(frame_atual, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Exibe o quadro atual com as detecções de movimento
        cv.imshow("Detecção de Movimento", frame_atual)

        # Atualiza o frame anterior para o próximo loop
        frame_anterior = frame_atual_gray

        # Sai do loop se a tecla 'q' for pressionada ou se a janela for fechada
        if cv.waitKey(30) & 0xFF == ord('q') or cv.getWindowProperty('Detecção de Movimento', cv.WND_PROP_VISIBLE) < 1:
            break

    captura.release()
    cv.destroyAllWindows()

# Exemplo de uso da função
detectar_movimento('../videos/parkour.mp4')
